[![Notebook Tutorial](__code/__all/notebook_tutorial.png)](https://neutronimaging.pages.ornl.gov/en/tutorial/notebooks/water_intake_profile_calculator/)

<img src='__docs/__all/notebook_rules.png' />

# Select your IPTS 

In [ ]:
from __code.ui_builder import UiBuilder
o_builder = UiBuilder(ui_name = 'ui_water_intake_profile.ui')
from __code.roi_selection_ui import Interface

from __code import system
from __code.water_intake_profile_calculator import WaterIntakeProfileCalculator, WaterIntakeProfileSelector

system.System.select_working_dir()
from __code.__all import custom_style
custom_style.style()

# Python Import 

In [ ]:
%gui qt

# Select Images to Process

In [ ]:
o_water = WaterIntakeProfileCalculator(working_dir=system.System.get_working_dir())
o_water.select_data()

# Select Profile Region 

In [ ]:
o_gui = WaterIntakeProfileSelector(dict_data=o_water.dict_files)
o_gui.show()